# fieldset_compare

compare the vector fields of different fieldsets

mainly used for checking the differences between the plume tracker OI data vs the thredds data

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from datetime import timedelta
import math

import cartopy
import cartopy.crs as ccrs
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from parcels import FieldSet, ParticleSet, JITParticle, AdvectionRK4, ErrorCode, ParticleFile
import scipy.spatial

import pyplume.utils as utils
from pyplume.dataloaders import DataLoader, dataset_to_fieldset
from pyplume.plotting import get_carree_axis, get_carree_gl, plot_vectorfield
from pyplume.constants import FIELD_NETCDF_DIR

In [ ]:
filenames = [
    FIELD_NETCDF_DIR / "tj_plume_1km_2022-09.nc",
    FIELD_NETCDF_DIR / "oi_fields/Tot_SDLJ_202209.nc"
]

files = []
for fname in filenames:
    ds = DataLoader(fname).dataset
    fs = dataset_to_fieldset(ds)
    files.append({"ds": ds, "fs": fs})

domain = {
    "S": 32.525,
    "N": 32.7,
    "W": -117.27,
    "E": -117.09
}

# if lat and lon dimensions are the same, the stuff will run MUCH quicker
dimensions_same = True

### comparing fieldset vectors

takes the first vector field, and checks each vector. finds the closest vector in the second vector field, and finds their angle and magnitude difference

note it might take a few minutes to run

In [ ]:
# set up kd trees/methods in order to process fields that do not share the same coordinates
for f in files:
    f["latkdtree"] = scipy.spatial.cKDTree(np.array([f["ds"]["lat"]]).T)
    f["lonkdtree"] = scipy.spatial.cKDTree(np.array([f["ds"]["lon"]]).T)

def get_nearest_index(ref, lat, lon):
    return ref["latkdtree"].query([lat])[1], ref["lonkdtree"].query([lon])[1]

In [ ]:
overlap_times = np.intersect1d(files[0]["ds"]["time"], files[1]["ds"]["time"])

# kind of hardcoded
ang_diff = np.full(files[0]["ds"]["U"].shape, np.nan, dtype=np.float32)
mag_diff = np.full(ang_diff.shape, np.nan, dtype=np.float32)

if not dimensions_same:
    for t, i, j in np.ndindex(ang_diff.shape):
        time = files[0]["ds"]["time"].values[t]
        u = files[0]["ds"]["U"].values[t, i, j]
        if np.isnan(u) or time not in files[1]["ds"]["time"]:
            continue
        # OI data is missing some time values
        t_other = np.where(files[1]["ds"]["time"].values == time)[0]
        if t_other.size == 0:
            continue
        else:
            t_other = t_other[0]
        lat = files[0]["ds"]["lat"][i].values
        lon = files[0]["ds"]["lon"][j].values
        lat_ind, lon_ind = get_nearest_index(files[1], lat, lon)
    #     print(utils.haversine(lat, files[1]["ds"]["lat"].values[lat_ind], lon, files[1]["ds"]["lon"].values[lon_ind]))
        if abs(lat - files[1]["ds"]["lat"].values[lat_ind]) > 0.00903 or abs(lon - files[1]["ds"]["lon"].values[lon_ind]) > 0.0107:
            continue
        v = files[0]["ds"]["V"].values[t, i, j]
        u_other = files[1]["ds"]["U"].values[t_other, lat_ind, lon_ind]
        v_other = files[1]["ds"]["V"].values[t_other, lat_ind, lon_ind]
        mag = np.sqrt(u * u + v * v)
        mag_other = np.sqrt(u_other * u_other + v_other * v_other)
        dot_mag_rat = (u * u_other + v * v_other) / (mag * mag_other)
        # rounding errors cause arccos to go out of domain
        if dot_mag_rat > 1:
            ang = 0
        elif dot_mag_rat < -1:
            ang = 180
        else:
            ang = np.arccos(dot_mag_rat) * 180 / math.pi
        ang_diff[t, i, j] = ang
        mag_diff[t, i, j] = abs(mag - mag_other)
else:
    # runs significantly quicker (please use this)
    u_shape = files[0]["ds"]["U"].shape
    ang_diff = np.full((len(overlap_times), u_shape[1], u_shape[2]), np.nan, dtype=np.float32)
    mag_diff = np.full(ang_diff.shape, np.nan, dtype=np.float32)
    for i, time in enumerate(overlap_times):
        vec_diff_dir = utils.get_dir("results/fs_comp_vectors")
        fig = plt.figure()
        fig.patch.set_facecolor("w")
        fig.set_size_inches(12, 5)
        fig.tight_layout()
        plot_vectorfield(files[0]["ds"], show_time=time, vmin=0, vmax=1, fig=fig, pos=121, titlestr="Thredds")
        plot_vectorfield(files[1]["ds"], show_time=time, vmin=0, vmax=1, fig=fig, pos=122, titlestr="OI")
        strtime = str(time.astype('datetime64[s]')).replace(":", "-")
        fig.savefig(vec_diff_dir / f"field_comp_{strtime}.png")
        plt.close(fig)

        tidx0 = np.where(files[0]["ds"]["time"] == time)[0][0]
        tidx1 = np.where(files[1]["ds"]["time"] == time)[0][0]

        u0 = files[0]["ds"]["U"].values[tidx0]
        v0 = files[0]["ds"]["V"].values[tidx0]
        u1 = files[1]["ds"]["U"].values[tidx1]
        v1 = files[1]["ds"]["V"].values[tidx1]

        mag0 = np.sqrt(u0 ** 2 + v0 ** 2)
        mag1 = np.sqrt(u1 ** 2 + v1 ** 2)
        dot_mag_rat = (u0 * u1 + v0 * v1) / (mag0 * mag1)
        # rounding errors cause arccos to go out of domain
        dot_mag_rat[np.where(dot_mag_rat > 1)] = 1
        dot_mag_rat[np.where(dot_mag_rat < -1)] = -1
        ang = np.arccos(dot_mag_rat) * 180 / math.pi
        ang_diff[i] = ang
        mag_diff[i] = abs(mag0 - mag1)

In [ ]:
np.nanmean(ang_diff), np.nanmedian(ang_diff), np.nanmax(ang_diff)

In [ ]:
np.nanmean(mag_diff), np.nanmedian(mag_diff), np.nanmax(mag_diff)

In [ ]:
ang_means = np.nanmean(ang_diff, axis=0)
mag_means = np.nanmean(mag_diff, axis=0)

distribution of measure differences

In [ ]:
plt.hist(ang_diff[np.where(~np.isnan(ang_diff))].flatten())
plt.title("angle differences (degrees)")
plt.show()
plt.hist(mag_diff[np.where(~np.isnan(mag_diff))].flatten())
plt.title("magnitude differences (m/s)")
plt.show()

generate heatmaps of measure differences

In [ ]:
lon, lat = np.meshgrid(files[0]["ds"]["lon"].values, files[0]["ds"]["lat"].values)
first_time = np.datetime64(files[0]["ds"]["time"][0].values, "M")
deg_diff_dir = utils.get_dir("results/fs_comp_angle")

fig, ax = get_carree_axis(domain)
gl = get_carree_gl(ax)
plt.contourf(lon, lat, ang_means, np.linspace(0, np.nanmax(ang_means), 30), cmap='inferno')
plt.title(f"Avg vector degree diff ({first_time})")
plt.colorbar()
plt.savefig(deg_diff_dir / f"heat_deg_{first_time}")
plt.show()

In [ ]:
deg_diff_dir = utils.get_dir("results/fs_comp_mag")

fig, ax = get_carree_axis(domain)
gl = get_carree_gl(ax)
plt.contourf(lon, lat, mag_means, np.linspace(0, np.nanmax(mag_means), 20), cmap='inferno')
plt.title(f"Avg magnitude diff (m/s) ({first_time})")
plt.colorbar()
plt.savefig(deg_diff_dir / f"heat_mag_{first_time}")
plt.show()